In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
from fredapi import Fred



In [3]:
df = pd.read_csv("data.csv", sep=";", decimal=",")
df = df.rename(columns={
    "Column1": "Date",
    "Column2": "SPX",
    "Column3": "S5SFTW",
    "Column4": "S5PHRM",
    "Column5": "S5CPGS",
    "Column6": "S5ENRSX",
    "Column7": "S5FDBT",
    "Column8": "S5TECH",
    "Column9": "S5RETL",
    "Column10": "S5BANKX",
    "Column11": "S5HCES",
    "Column12": "S5DIVF",
    "Column13": "S5UTILX",
    "Column14": "S5MEDA",
    "Column15": "S5REAL",
    "Column16": "S5TELSX",
    "Column17": "S5MATRX",
    "Column18": "S5INSU",
    "Column19": "S5FDSR",
    "Column20": "S5HOUS",
    "Column21": "S5SSEQX",
    "Column22": "S5TRAN",
    "Column23": "S5HOTR",
    "Column24": "S5CODU",
    "Column25": "S5AUCO",
    "Column26": "S5COMS",
})
df["Date"] = pd.to_datetime(df["Date"], format="%d/%m/%Y")

In [4]:
def GetReturn(df,date,lookback=180):
    date=pd.to_datetime(date)
    if date not in df["Date"].values:#add breaker if windows not in df
        raise ValueError("Date not in dataframe")
    returns_df = df[["Date","S5SFTW","S5PHRM","S5CPGS","S5ENRSX","S5FDBT","S5TECH","S5RETL","S5BANKX","S5HCES","S5DIVF","S5UTILX","S5MEDA","S5REAL","S5TELSX","S5MATRX","S5INSU","S5FDSR","S5HOUS","S5SSEQX","S5TRAN","S5HOTR","S5CODU","S5AUCO","S5COMS"]].copy()

    date_list=returns_df.drop(columns="Date")
    date_index = returns_df.index[returns_df["Date"] == date][0]
    returns_df=returns_df[(returns_df.index<=date_index) & (returns_df.index>=date_index-lookback) ]
    returns_df.drop(columns="Date",inplace=True)

    returns_df = np.log(returns_df/ returns_df.shift(1))
    returns_df.dropna(inplace=True)
    #print(returns_df.std().mean()) #verification if std is around 1% daily

    return returns_df


def GetReturnSPX(df,date,lookback=180):
    date=pd.to_datetime(date)
    if date not in df["Date"].values:#add breaker if windows not in df
        raise ValueError("Date not in dataframe")
    returns_df = df[["Date","SPX"]].copy()

    date_list=returns_df.drop(columns="Date")
    date_index = returns_df.index[returns_df["Date"] == date][0]
    returns_df=returns_df[(returns_df.index<=date_index) & (returns_df.index>=date_index-lookback) ]
    returns_df.drop(columns="Date",inplace=True)

    returns_df = np.log(returns_df/ returns_df.shift(1))
    returns_df.dropna(inplace=True)
    #print(returns_df.std().mean()) #verification if std is around 1% daily

    return returns_df

#Returns=GetReturn(df,"2020-05-11",lookback=180)
#ReturnsSPX=GetReturnSPX(df,"2020-05-11",lookback=180)

In [5]:
def GetSigma(df,date,lookback=180):
    returns_df=GetReturn(df,date,lookback=lookback)
    #covariance matric from returns_df
    sigma_windowed=returns_df.cov()

    return sigma_windowed

#Sigma=GetSigma(df,"2020-05-11",lookback=180)

In [6]:
def GetRfDataframe(df):
    fred = Fred(api_key="5c742a53d96bd3085e9199dcdb5af60b")
    riskfree = fred.get_series('DFF')
    # riskfree = fred.get_series('DTB1MO')

    riskfree = riskfree.to_frame(name='FedFunds')
    riskfree.index.name = "Date"
    riskfree = riskfree[riskfree.index >= "2002-01-01"]
    riskfree["FedFunds"]=riskfree["FedFunds"]/100
    list_days_open = pd.to_datetime(df["Date"], dayfirst=True, errors="coerce")
    list_days_full = pd.to_datetime(riskfree.index, dayfirst=True, errors="coerce")

    list_days_open=[pd.to_datetime(date) for date in list_days_open]
    list_days_full=[pd.to_datetime(date) for date in list_days_full]


    list_days_open_pondered=[]
    riskfree_list=[]
    count_list=[]
    timestamp=0
    while timestamp < len(list_days_full)-1:

      if list_days_full[timestamp+1] in list_days_open:
            list_days_open_pondered.append(list_days_full[timestamp])
            riskfree_list.append(riskfree["FedFunds"].loc[list_days_full[timestamp]])
            count_list.append(1)
            timestamp += 1

      else:
          count = 0
          timestampbis = timestamp
          while (timestamp + 1 < len(list_days_full)) and (list_days_full[timestamp + 1] not in list_days_open):
              timestamp += 1
              count += 1

          list_days_open_pondered.append(list_days_full[timestampbis])  # jour de départ
          riskfree_list.append(riskfree["FedFunds"].loc[list_days_full[timestampbis]])
          count_list.append(count+1)
          timestamp += 1

    RfDf=pd.DataFrame({"Date":list_days_open_pondered,"Rf":riskfree_list,"Count":count_list})
    RfDf=RfDf.set_index("Date")
    return RfDf


def GetRiskFree(df,date,lookback=180):

    RfDf=GetRfDataframe(df)
    positionOfStartDate=df.index[df["Date"]==pd.to_datetime(date)][0]-lookback
    startDate=pd.to_datetime(df.iloc[positionOfStartDate,0])

    endDate=pd.to_datetime(date)
    RfDf=RfDf[(RfDf.index >= startDate) & (RfDf.index <= endDate )]
    CumulativeRf=[]

    for i in range(len(RfDf)):
      if i==0:
        CumulativeRf.append(pow((1+RfDf["Rf"].iloc[i]),(RfDf["Count"].iloc[i]/360)))
      else:
        CumulativeRf.append(pow((1+RfDf["Rf"].iloc[i]),(RfDf["Count"].iloc[i]/360))*CumulativeRf[i-1])

    RfDf["CumulativeRf"]=CumulativeRf
    RfDf["CumulativeRf"]= RfDf["CumulativeRf"]-1

    return RfDf["CumulativeRf"].iloc[-1]

#RiskFree=GetRiskFree(df,"2020-05-11",lookback=180)

In [7]:
def GetWeight(df,date):
    #for the moment we will use the equal weight
    weight_vector=np.zeros((24,1))
    for i in range(0,24):
        weight_vector[i]=1/24

    return weight_vector
#Weight=GetWeight(df,"2020-05-11")


In [8]:
def GetLambda(df,date,lookback=180):
    returns=GetReturn(df,date,lookback)
    returns=returns+1

    avg_return=returns.prod()-1 #geometric 180days return
    weight_vector=GetWeight(df=0,date=0)

    Sigma=GetSigma(df,date,lookback=180)

    var = float(weight_vector.T @ Sigma.values @ weight_vector)
    lambda_value=(avg_return@weight_vector - GetRiskFree(df,date,lookback=180))/np.sqrt(var)
    return lambda_value


#Lambda=GetLambda(df,"2016-05-11",lookback=180)


In [31]:
def GetPMatrix(df,date, lookback,longonly=False,proportion=3,offset=3):
    AssetColumns=["S5SFTW","S5PHRM","S5CPGS","S5ENRSX","S5FDBT","S5TECH","S5RETL","S5BANKX","S5HCES","S5DIVF","S5UTILX","S5MEDA","S5REAL","S5TELSX","S5MATRX","S5INSU","S5FDSR","S5HOUS","S5SSEQX","S5TRAN","S5HOTR","S5CODU","S5AUCO","S5COMS"]
    bestperformer = []
    worstperformer = []
    performerc = []
    returnBestPerformer=[]
    returnWorstPerformer=[]
    endDateIndex=df.index[df["Date"]==pd.to_datetime(date)][0]
    startDateIndex=df.index[df["Date"]==pd.to_datetime(date)][0]-lookback

    for i in range(1, df.shape[1]):  #loop through asset columns
        performerc.append((((float(df.iloc[endDateIndex, i]) / float(df.iloc[startDateIndex, i]) - 1) * 100), i - 1))

    performerc.sort(reverse=True)
    for i in range(proportion):
        bestperformer.append(performerc[i][1])
        returnBestPerformer.append(performerc[i][0])

    for i in range(len(performerc) - offset - proportion, len(performerc) - offset):
        if longonly==False:
            worstperformer.append(performerc[i][1])
            returnWorstPerformer.append(performerc[i][0])

    P=np.zeros((1,24))
    if longonly==True:
        for i in range(len(AssetColumns)):
            P[0,i]=-1/(24-proportion)
    else :
        for i in range(len(AssetColumns)):
            P[0,i]=0

    for i in range(len(AssetColumns)):
        if i in bestperformer:
            P[0,i]=1/proportion
        elif i in worstperformer and longonly==False:
            P[0,i]=-1/proportion


    if len(returnWorstPerformer)==0:
        returnWorstPerformer.append(0)

    spreadLoosersWinnners=np.mean(returnBestPerformer)-np.mean(returnWorstPerformer)
    Q=np.array([[spreadLoosersWinnners/100]]) #convert to decimal
    return P, Q

#PMatrix,TempoQ=PMatrix(df,"2016-05-11",lookback=180)

In [10]:
def GetOmega(PMatrix, Sigma, c=0.99):
    #Omega is the uncertainty of the views
    factorC=(1/c-1)
    Omega=factorC*PMatrix@Sigma@np.transpose(PMatrix)


    return Omega

In [33]:
def BlackAndLittermanModel(backtestStartDate, rebalancingFrequency, lookbackPeriod, df):
    #implement the full backtest of the black and litterman model

    #---------
    #PARAMETERS
    #---------

    free_asset=0 #proportion of risk free asset allocated in the benchmark
    taux=0.01


    Sigma=GetSigma(df,backtestStartDate,lookback=lookbackPeriod)
    Lambda=GetLambda(df,backtestStartDate,lookback=lookbackPeriod)
    PMatrix,Q= GetPMatrix(df,backtestStartDate, lookback=lookbackPeriod,longonly=True,proportion=3,offset=3)
    Omega=GetOmega(PMatrix, Sigma, c=0.5)
    rf=GetRiskFree(df,backtestStartDate,lookback=lookbackPeriod)
    weights = GetWeight(df, backtestStartDate)
    weights = np.array(weights).reshape(-1, 1)
    uimplied = Lambda * (Sigma @ weights) + rf





    optimizedReturn=(np.linalg.inv(np.linalg.inv(taux*Sigma)+np.transpose(PMatrix)@np.linalg.inv(Omega)@PMatrix)) @ (np.linalg.inv(taux*Sigma)@uimplied+np.transpose(PMatrix)@np.linalg.inv(Omega)@Q)

    print("BL Returns",optimizedReturn)

    #MarkowitzAllocation

    WeightBL=np.linalg.inv(Sigma)@(optimizedReturn-rf)/Lambda

    print("BL Weights",WeightBL)
    print("RF Weights",np.sum(WeightBL))















    pass

BlackAndLittermanModel("2016-05-11", rebalancingFrequency=3, lookbackPeriod=180, df=df)





C:\Users\mbard\AppData\Local\Temp\ipykernel_30480\4279076909.py:10: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  var = float(weight_vector.T @ Sigma.values @ weight_vector)


[[-0.04761905 -0.04761905 -0.04761905 -0.04761905 -0.04761905  0.33333333
  -0.04761905 -0.04761905 -0.04761905 -0.04761905 -0.04761905  0.33333333
  -0.04761905 -0.04761905 -0.04761905 -0.04761905 -0.04761905 -0.04761905
   0.33333333 -0.04761905 -0.04761905 -0.04761905 -0.04761905 -0.04761905]]
BL Returns            0
0   0.047301
1   0.025796
2   0.030325
3   0.058077
4  -0.003028
5   0.145508
6   0.018186
7   0.041472
8   0.007828
9   0.043991
10 -0.022732
11  0.077469
12 -0.002259
13  0.010344
14  0.044479
15  0.018046
16  0.004182
17 -0.007445
18  0.127067
19  0.042109
20  0.025319
21  0.005591
22  0.057820
23  0.026069
BL Weights              0
0   -28.325068
1   -28.325068
2   -28.325068
3   -28.325068
4   -28.325068
5   198.608811
6   -28.325068
7   -28.325068
8   -28.325068
9   -28.325068
10  -28.325068
11  198.608811
12  -28.325068
13  -28.325068
14  -28.325068
15  -28.325068
16  -28.325068
17  -28.325068
18  198.608811
19  -28.325068
20  -28.325068
21  -28.325068
22  -28.32

C:\Users\mbard\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\_core\fromnumeric.py:84: FutureWarning: The behavior of DataFrame.sum with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)
